# XGBOOST - GRIDSEARCH

In [1]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from datetime import datetime
from time import time

### GLOBAL VARIABLES

In [26]:
DATAPATH = 'data/train_test/'
SEED = 47
NITER = 100
CV = 5
SCORE = 'balanced_accuracy'
handlingnull = False
NJOBS = 4
USEGPU = False
NCLASS = 3 # number class to predict (if bivar set 0)

### LOAD DATASET

In [3]:
train_features = np.load(DATAPATH+'X_wavelet_discrete_train.npy')

In [4]:
train_labels = np.load(DATAPATH+'y_wavelet_discrete_train.npy')

In [5]:
train_features.shape

(1225, 19746)

In [7]:
### create a DMatrix and handling Null values
if handlingnull:
    #train_features[np.isnan(train_features)] = -9999
    xgtrain = xgb.DMatrix(train_features, train_labels, missing=-9999)
else:
    xgtrain = xgb.DMatrix(train_features, train_labels)

In [8]:
if handlingnull:
    train_features[np.isnan(train_features)] = -9999

### TRAIN MODEL

#### Set hyperparameters

In [9]:
# ======== General Parameters ======= #

# Select the type of model to run at each iteration. gbtree or gblinear.
booster = 'gbtree'


# Control the balance of positive and negative weights, useful for unbalanced classes. 
# A typical value to consider: sum(negative instances) / sum(positive instances)scale_pos_weight = 1
scale_pos_weight = [5, 1, 2] #int((len(train_labels) - np.sum(train_labels))/np.sum(train_labels))


# Learning Task Parameters

# This defines the loss function to be minimized. 
# - binary:logistic –logistic regression for binary classification, returns predicted probability (not class)
# - multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities)
#   you also need to set an additional num_class (number of classes) parameter defining the number of unique classes
# - multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class.
objective  = 'multi:softprob'


# The metric to be used for validation data.
# - rmse – root mean square error
# - mae – mean absolute error
# - logloss – negative log-likelihood
# - error – Binary classification error rate (0.5 threshold)
# - merror – Multiclass classification error rate
# - mlogloss – Multiclass logloss
# - auc: Area under the curve
eval_metric = 'mlogloss'

#### Load hyperparameters

In [10]:
xgb_params = np.load('output/hyperparameters/rseach_xgboost_classifier_bestparams_d2019-11-09.npy', allow_pickle=True).tolist()

In [11]:
xgb_params['seed'] = SEED
xgb_params['booster'] = booster
xgb_params['objective'] = objective
xgb_params['eval_metric'] = eval_metric
xgb_params['num_threads'] = NJOBS
xgb_params['num_class'] = 3
xgb_params['verbose'] = 0
xgb_params['scale_pos_weight'] = scale_pos_weight

In [12]:
if USEGPU:
    xgb_params['tree_method'] = 'gpu_hist'
    xgb_params['gpu_id'] = 0

In [13]:
xgb_params

{'subsample': 0.7,
 'reg_lambda': 0.5,
 'reg_alpha': 0.05,
 'min_child_weight': 1,
 'max_depth': 9,
 'learning_rate': 0.01,
 'gamma': 0.4,
 'colsample_bytree': 0.8,
 'seed': 47,
 'booster': 'gbtree',
 'objective': 'multi:softprob',
 'eval_metric': 'mlogloss',
 'num_threads': 7,
 'num_class': 3,
 'verbose': 0,
 'scale_pos_weight': [5, 1, 2]}

#### Find num boost

In [14]:
early_stopping_rounds = 50

In [15]:
cvresult = xgb.cv(xgb_params, xgtrain, 
                  num_boost_round = 1000, 
                  nfold = CV, 
                  metrics = eval_metric, 
                  early_stopping_rounds = early_stopping_rounds, 
                  seed = SEED)

[21:34:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 2 pruned nodes, max_depth=9
[21:34:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 2 pruned nodes, max_depth=9
[21:34:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 4 pruned nodes, max_depth=7
[21:34:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=9
[21:34:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 4 pruned nodes, max_depth=9
[21:34:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 2 pruned nodes, max_depth=9
[21:34:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 2 pruned nodes, max_depth=9
[21:34:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 2 pruned nodes, max_depth=9
[21:34:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 4 pruned nodes, max_

[21:34:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 2 pruned nodes, max_depth=9
[21:34:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 2 pruned nodes, max_depth=9
[21:34:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=9
[21:34:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=9
[21:34:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=9
[21:34:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=9
[21:34:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=8
[21:34:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=9
[21:34:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_

[21:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=9
[21:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 4 pruned nodes, max_depth=8
[21:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 2 pruned nodes, max_depth=9
[21:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 4 pruned nodes, max_depth=9
[21:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=9
[21:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 2 pruned nodes, max_depth=9
[21:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=9
[21:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=8
[21:34:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 4 pruned nodes, max_

[21:35:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 6 pruned nodes, max_depth=9
[21:35:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 6 pruned nodes, max_depth=8
[21:35:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=9
[21:35:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=8
[21:35:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 4 pruned nodes, max_depth=9
[21:35:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=9
[21:35:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=9
[21:35:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 2 pruned nodes, max_depth=9
[21:35:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_

[21:35:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 4 pruned nodes, max_depth=9
[21:35:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 4 pruned nodes, max_depth=9
[21:35:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 2 pruned nodes, max_depth=8
[21:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=9
[21:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=7
[21:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 4 pruned nodes, max_depth=9
[21:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 4 pruned nodes, max_depth=7
[21:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 2 pruned nodes, max_depth=8
[21:35:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 2 pruned nodes, max_

...

[21:44:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 8 pruned nodes, max_depth=5
[21:44:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=6
[21:44:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 16 pruned nodes, max_depth=6
[21:44:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 12 pruned nodes, max_depth=6
[21:44:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 18 pruned nodes, max_depth=5
[21:44:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 12 pruned nodes, max_depth=6
[21:44:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=5
[21:44:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 14 pruned nodes, max_depth=5
[21:44:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 14 pruned node

[21:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 14 pruned nodes, max_depth=5
[21:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 14 pruned nodes, max_depth=6
[21:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 14 pruned nodes, max_depth=6
[21:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 16 pruned nodes, max_depth=5
[21:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 8 pruned nodes, max_depth=7
[21:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 8 pruned nodes, max_depth=5
[21:44:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 8 pruned nodes, max_depth=7
[21:44:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 16 pruned nodes, max_depth=5
[21:44:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes,

[21:45:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 10 pruned nodes, max_depth=5
[21:45:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 14 pruned nodes, max_depth=5
[21:45:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 20 pruned nodes, max_depth=6
[21:45:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 12 pruned nodes, max_depth=4
[21:45:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 14 pruned nodes, max_depth=5
[21:45:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 18 pruned nodes, max_depth=6
[21:45:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=5
[21:45:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 8 pruned nodes, max_depth=8
[21:45:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 14 pruned node

[21:45:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 12 pruned nodes, max_depth=7
[21:45:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[21:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 16 pruned nodes, max_depth=5
[21:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 12 pruned nodes, max_depth=5
[21:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 20 pruned nodes, max_depth=5
[21:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 14 pruned nodes, max_depth=6
[21:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 12 pruned nodes, max_depth=8
[21:45:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=6
[21:45:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 12 pruned no

[21:45:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=6
[21:45:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 8 pruned nodes, max_depth=5
[21:45:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 14 pruned nodes, max_depth=4
[21:45:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 10 pruned nodes, max_depth=6
[21:45:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 8 pruned nodes, max_depth=6


In [16]:
n_estimators = cvresult.shape[0]

In [17]:
print("Best number of boosters: ", n_estimators)

Best number of boosters:  311


#### Set hyperparameters

In [19]:
# ======== Booster Parameters ======== # 

# Analogous to learning rate in GBM. 
# Typical final values to be used: 0.01-0.2
eta = [0.01] 

# Defines the minimum sum of weights of all observations required in a child.
min_child_weight = [xgb_params['min_child_weight']]

# The maximum depth of a tree
max_depth = [i for i in range(int(xgb_params['max_depth']) - 1, int(xgb_params['max_depth']) + 2, 1)]

# A node is split only when the resulting split gives a positive reduction in the loss function. 
# Gamma specifies the minimum loss reduction required to make a split.
gamma = [xgb_params['gamma']]

# Denotes the fraction of observations to be randomly samples for each tree.
subsample = [i/100 for i in range(int(xgb_params['subsample']*100) - 4, int(xgb_params['subsample']*100) + 6, 3)]

# Denotes the fraction of columns to be randomly samples for each tree.
colsample_bytree = [i/100 for i in range(int(xgb_params['colsample_bytree']*100) - 4, int(xgb_params['colsample_bytree']*100) + 6, 3)]

# L2 regularization term on weights (analogous to Ridge regression)
reg_lambda =  [i/100 for i in range(int(xgb_params['reg_lambda']*100) - 4, int(xgb_params['reg_lambda']*100) + 4, 4)]

# L1 regularization term on weight (analogous to Lasso regression)
reg_alpha = [xgb_params['reg_alpha']]

[xgboost params](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

In [20]:
# Create the random grid
param_grid = {
    'learning_rate' : eta,
    'min_child_weight' : min_child_weight,
    'max_depth' : max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree' : colsample_bytree,
    'reg_lambda' : reg_lambda,
    'reg_alpha' : reg_alpha,
}

In [21]:
param_grid

{'learning_rate': [0.01],
 'min_child_weight': [1],
 'max_depth': [8, 9, 10],
 'gamma': [0.4],
 'subsample': [0.66, 0.69, 0.72, 0.75],
 'colsample_bytree': [0.76, 0.79, 0.82, 0.85],
 'reg_lambda': [0.46, 0.5],
 'reg_alpha': [0.05]}

In [22]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model = XGBClassifier(n_estimators=n_estimators, 
                      scale_pos_weight=scale_pos_weight, 
                      num_class=NCLASS, 
                      objective=objective)

In [23]:
if USEGPU:
    model.set_params(gpu_id = 0)
    model.set_params(tree_method='gpu_hist')

In [27]:
# Random search of parameters, using CV fold cross validation, 
# search across NITER different combinations, and use all available cores
xgboost_gsearch = GridSearchCV(estimator = model, param_grid = param_grid, scoring=SCORE, cv = CV, verbose=1,  n_jobs = NJOBS)# Fit the random search model


#### Training

In [ ]:
start = time()
xgboost_gsearch.fit(train_features, train_labels)
print("GridSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), NITER))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 183.5min


#### Saving results

In [ ]:
cv_results = pd.DataFrame(xgboost_gsearch.cv_results_)

In [ ]:
cv_results.to_csv('output/results/gsearch_xgboost_classifier_d' + str(datetime.now().date()) + '.csv',sep=';',index=False)

#### Best estimator

In [1]:
xgboost_gsearch.best_estimator_

NameError: name 'xgboost_gsearch' is not defined

#### Best parameter

In [71]:
best_params = xgboost_gsearch.best_params_

{'colsample_bytree': 0.72,
 'gamma': 0.1,
 'learning_rate': 0.01,
 'max_depth': 9,
 'min_child_weight': 3,
 'reg_alpha': 0.001,
 'reg_lambda': 0.9,
 'subsample': 0.85}

In [ ]:
best_params['n_estimators'] = n_estimators

#### Best Score

In [78]:
print(SCORE,' : ', xgboost_gsearch.best_score_)

balanced_accuracy  :  0.7670268843746242


#### Saving best model

In [80]:
np.save('output/hyperparameters/gseach_xgboost_classifier_bestparams_d' + str(datetime.now().date()) + '.npy', best_params)

In [81]:
np.save('output/results/gseach_xgboost_classifier_best_estimator_d' + str(datetime.now().date()) + '.npy', xgboost_gsearch.best_estimator_)